In [18]:
# initialize model and print out the statistics yaml
import json
from IPython.display import display, Markdown
import yaml

import torch.nn as nn

from mymodel import init_my_model


model = init_my_model() 
min_max_yaml = yaml.safe_load(open("min_max.yaml", "r"))
config = json.load(open("config.json", "r"))

total_params = sum(p.numel() for p in model.parameters())
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)

model_name = model.__class__.__name__
num_layers = len(list(model.net.children()))


# Model Architecture

In [19]:
md_text = f"The following section describes the architecture of the {model_name} used for the submission. "
md_text += f"The {model_name} has a total number of {total_params} parameters "
md_text += f"with {trainable_params} trainable parameters. "
md_text += f"It consists of {num_layers} layers. "
md_text += "The layers are structured as follows:\n\n"
md_text += "| Index | Layer Type | Details |\n"
md_text += "| :--- | :--- | :--- |\n"

for i, layer in enumerate(model.net.children()):
    layer_type = layer.__class__.__name__
    
    if isinstance(layer, nn.Conv2d):
        details = f"In: {layer.in_channels}, Out: {layer.out_channels}, Kernel: {layer.kernel_size[0]}x{layer.kernel_size[1]}"
    elif isinstance(layer, nn.LeakyReLU):
        details = f"Negative Slope: {layer.negative_slope}"
    else:
        details = "---"
        
    md_text += f"| {i} | {layer_type} | {details} |\n"

display(Markdown(md_text))

The following section describes the architecture of the FluidCNN used for the submission. The FluidCNN has a total number of 65170 parameters with 65170 trainable parameters. It consists of 13 layers. The layers are structured as follows:

| Index | Layer Type | Details |
| :--- | :--- | :--- |
| 0 | Conv2d | In: 1, Out: 16, Kernel: 7x7 |
| 1 | LeakyReLU | Negative Slope: 0.01 |
| 2 | Conv2d | In: 16, Out: 16, Kernel: 7x7 |
| 3 | LeakyReLU | Negative Slope: 0.01 |
| 4 | Conv2d | In: 16, Out: 16, Kernel: 7x7 |
| 5 | LeakyReLU | Negative Slope: 0.01 |
| 6 | Conv2d | In: 16, Out: 16, Kernel: 7x7 |
| 7 | LeakyReLU | Negative Slope: 0.01 |
| 8 | Conv2d | In: 16, Out: 16, Kernel: 7x7 |
| 9 | LeakyReLU | Negative Slope: 0.01 |
| 10 | Conv2d | In: 16, Out: 16, Kernel: 7x7 |
| 11 | LeakyReLU | Negative Slope: 0.01 |
| 12 | Conv2d | In: 16, Out: 2, Kernel: 7x7 |


In [23]:
md_content = "This was the configuration used to initialize the model and load the data:\n\n"
md_content += "| Parameter | Value |\n"
md_content += "| :--- | :--- |\n"

for k, v in config.items():
    display_key = k.replace("_", " ").capitalize()
    md_content += f"| **{display_key}** | {v} |\n"

display(Markdown(md_content))

This was the configuration used to initialize the model and load the data:

| Parameter | Value |
| :--- | :--- |
| **Epochs** | 5000 |
| **Batch size** | 16 |
| **Activation** | LeakyReLU |
| **Lr** | 5e-05 |
| **Num hidden layers** | 5 |
| **Kernel size** | 7 |
| **In channels** | 1 |
| **Out channels** | 2 |
| **Hidden channels** | 16 |
| **Output activation** | None |
| **Use bias** | True |
| **Padding mode** | zeros |
| **Random split** | False |


In [21]:
min_max_dict = min_max_yaml  

def format_dict_to_md(d, level=0):
    lines = []
    for k, v in d.items():
        indent = "  " * level
        if isinstance(v, dict):
            lines.append(f"{indent}* **{k.capitalize()}**:")
            lines.extend(format_dict_to_md(v, level + 1))
        else:
            val = f"{v:.4f}" if isinstance(v, (float)) else v
            lines.append(f"{indent}* **{k}**: {val}")
    return lines

md_list = "\n".join(format_dict_to_md(min_max_dict))

display(Markdown("The input data was then normalized based on:"))
display(Markdown(md_list))

The input data was then normalized based on:

* **Inputs**:
  * **U**:
    * **max**: 1.5000
    * **min**: 0.0000
* **Labels**:
  * **U**:
    * **max**: 1.5000
    * **min**: -0.1241
  * **V**:
    * **max**: 0.0918
    * **min**: -0.3537

# Evaluation

## Loss Graphs

The following figure shows the training, validation and test loss over the total number of epochs the model was trained on.

![Train and test losses](losses.png)

## Performance Metrics

Measured evaluation metrics of the best model for train, val, test RMSE, MAE on the un-normed predicted u, v fields

# Predictions

## Interpolation

Sample data point predictions and comparisons to their labels (and visualization of vector fields)

## Extrapolation

To see how well the model generalizes, the following section should contain three out-of-distribution test samples.